# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [5]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

Q = pd.read_csv("cost-Q.tsv", sep="\t", index_col=0)
R = pd.read_csv("cost-R.tsv", sep="\t", index_col=0)
A = pd.read_csv("model-A.tsv", sep="\t", index_col=0)
B = pd.read_csv("model-B.tsv", sep="\t", index_col=0)

A_matrix = A.to_numpy(dtype=float)
B_matrix = B.to_numpy(dtype=float)
Q_matrix = Q.to_numpy(dtype=float)
R_matrix = R.to_numpy(dtype=float)

def solve_discrete_are_iterative(A_matrix, B_matrix, Q_matrix, R_matrix,
                                max_iterations=20000, tolerance=0.000000000001):
    P = Q_matrix.copy()
    for _ in range(max_iterations):
        R_plus_BtPB = R_matrix + (B_matrix.T @ P @ B_matrix)
        K_temp = np.linalg.inv(R_plus_BtPB) @ (B_matrix.T @ P @ A_matrix)
        P_next = Q_matrix + (A_matrix.T @ P @ A_matrix) - (A_matrix.T @ P @ B_matrix @ K_temp)
        if np.max(np.abs(P_next - P)) < tolerance:
            return P_next
        P = P_next
    raise RuntimeError("DARE did not converge.")

P = solve_discrete_are_iterative(A_matrix, B_matrix, Q_matrix, R_matrix)
K = np.linalg.inv(R_matrix + (B_matrix.T @ P @ B_matrix)) @ (B_matrix.T @ P @ A_matrix)

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [6]:
# YOUR CHANGES HERE
K_df = pd.DataFrame(K, columns=["x", "v", "a"])
K_df.to_csv("control-K-intended.tsv", sep="\t", index=False, float_format="%.15f")

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [8]:
# YOUR CHANGES HERE
log = pd.read_csv("qc-log.tsv", sep="\t", index_col=0)
K_df = pd.read_csv("control-K-intended.tsv", sep="\t")
K_row = K_df[["x", "v", "a"]].to_numpy(dtype=float)

states = log[["x", "v", "a"]].to_numpy(dtype=float)
u_check = -(states @ K_row.T).reshape(-1)

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [9]:
# YOUR CHANGES HERE
qc_check_df = pd.DataFrame({"index": log.index.to_numpy(), "u_check": u_check})
qc_check_df.to_csv("qc-check.tsv", sep="\t", index=False, float_format="%.15f")

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [10]:
# YOUR CHANGES HERE

possible_action_columns = ["u", "u_t", "u_logged", "u_action"]
action_column = None
for name in possible_action_columns:
    if name in log.columns:
        action_column = name
        break

if action_column is None:
    raise ValueError(f"Could not find an action column. Columns are: {list(log.columns)}")

states = log[["x", "v", "a"]].to_numpy(dtype=float)
u_logged = log[action_column].to_numpy(dtype=float)

k_actual_column_vector, _, _, _ = np.linalg.lstsq(states, -u_logged, rcond=None)
K_actual = k_actual_column_vector.reshape(1, 3)

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [11]:
# YOUR CHANGES HERE
K_actual_df = pd.DataFrame(K_actual, columns=["x", "v", "a"])
K_actual_df.to_csv("control-K-actual.tsv", sep="\t", index=False, float_format="%.15f")

Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [14]:
# YOUR CHANGES HERE
train = pd.read_csv("qc-train.tsv", sep="\t", index_col=0)

X = train[["x", "v", "a"]].to_numpy(dtype=float)
u = train[["u"]].to_numpy(dtype=float)

X_t = X[:-1, :]
u_t = u[:-1, :]
X_tp1 = X[1:, :]

X_aug = np.hstack([X_t, u_t])

Theta, _, _, _ = np.linalg.lstsq(X_aug, X_tp1, rcond=None)

A_new = Theta[0:3, :].T
B_new = Theta[3, :].reshape(3, 1)


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [15]:
# YOUR CHANGES HERE
A_new_df = pd.DataFrame(A_new, index=["x", "v", "a"], columns=["x", "v", "a"])
B_new_df = pd.DataFrame(B_new, index=["x", "v", "a"], columns=["u"])

A_new_df.to_csv("model-A-new.tsv", sep="\t", float_format="%.15f")
B_new_df.to_csv("model-B-new.tsv", sep="\t", float_format="%.15f")


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.